## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import torch
import torch.nn as nn
from torch.nn.functional import gelu
from torch.nn import CrossEntropyLoss

from datasets import load_dataset, DatasetDict

from tokenizers import ByteLevelBPETokenizer
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

from transformers import (RobertaTokenizer, PreTrainedModel, RobertaConfig, 
                          RobertaForMaskedLM, DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)

from transformers.modeling_outputs import MaskedLMOutput

## Helper functions

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

## Paths

In [ ]:
data = Path("data/")
!ls {data}

oscar.eo.ds  oscar.eo.txt


In [ ]:
model_dir = "models/esperberto"
!ls {model_dir}

merges.txt  vocab.json


## Get data

In [ ]:
!wget -c -O data/oscar.eo.txt https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt

--2021-04-16 09:19:52--  https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt
Resolving cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)... 99.84.114.112, 99.84.114.24, 99.84.114.120, ...
Connecting to cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)|99.84.114.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 312733741 (298M) [text/plain]
Saving to: ‘data/oscar.eo.txt’

data/oscar.eo.txt   100%[===================>] 298.25M  56.4MB/s    in 5.3s    

2021-04-16 09:19:58 (56.3 MB/s) - ‘data/oscar.eo.txt’ saved [312733741/312733741]



In [ ]:
!head {data/"oscar.eo.txt"}

Ĉu ... preĝi | mediti | ricevi instigojn || kanti | muziki || informiĝi | legi | studi || prepari Diservon
Temas pri kolekto de kristanaj kantoj, eldonita de Adolf Burkhardt inter 1974 kaj 1990 en dek kajeretoj. Ili estas reeldonitaj inter 1995 kaj 1998 de Bernhard Eichkorn en tri kajeroj, kies tria estas pliampleksigita per Dek Novaj Kantoj kaj suplemento, same de Adolf Burkhardt.
En la dua kaj tria kajero oni adiciis 300 al la originaj kantonumeroj, por ke oni povu pli facile uzi la kajerojn kune kun la KELI-himnaro Adoru Kantante, kiu havas malpli ol 300 numerojn.
Ni ĝojus, se iu trovus bonajn ekzemplerojn de la dek originaj kajeretoj kaj tempon por skani ankaŭ ilin. Bonvolu ekkontaktiĝi kun ni!
Lerni Esperanton per telefono, novaĵoj Poŝtkarto 120 jaroj de fervojo Svitavy-Polička 189… T.n.migranta poŝtkarto el 1908 BK - Kongresa Biblioteko en Vaŝingtono 1- 910 BK - Nederlando- Esperanta elektra tramo en Hago (… La lernolibro "Esperanto per rekta metodo" jam en… IMG 7181 Nova poŝtkar

## Train tokenizer

In [ ]:
paths = [str(x) for x in data.glob("**/*.txt")]
paths

['data/oscar.eo.txt']

In [ ]:
tokenizer = ByteLevelBPETokenizer()

In [ ]:
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [ ]:
tokenizer.save_model(model_dir)

['models/esperberto/vocab.json', 'models/esperberto/merges.txt']

In [ ]:
tokenizer = ByteLevelBPETokenizer(
    f"{model_dir}/vocab.json",
    f"{model_dir}/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
tokenizer.encode("Mi estas Julien.")

Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
tokenizer.encode("Mi estas Julien.").tokens

['<s>', 'Mi', 'Ġestas', 'ĠJuli', 'en', '.', '</s>']

## Load data

In [ ]:
ds = load_dataset('text', data_files={'train': [paths[0]]})
ds

Using custom data configuration default-31220d7f73477105
Reusing dataset text (/root/.cache/huggingface/datasets/text/default-31220d7f73477105/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 974616
    })
})

In [ ]:
show_random_elements(ds["train"])

,text
0,"Ĉar ludantoj devas pripensi movadon de siaj pecoj, la planedoj kaj la suno, ne nur laŭ rektaj linioj, sed ankaŭ laŭ elipsoj kaj cirkloj, bonvolu zorge legi la instrukciojn."
1,"La internacia komunumo: la ""nova epoko"" povas esti enirpunkto por kompreni la 19an Tutlandan Kongreson de KPĈ"
2,Estos definitive rekomendas amikoj kiujn ni opinias ke ĝi estas la plej bona enreta moveblaj kazino ejoj kiu estas sekura kaj amuza samtempe! Do vizitu ĉi ludo online paĝaro hodiaŭ!
3,"- HONORA MEDALO al aŭtoro donita de Societo Nacia de Kuraĝigo al Bonfaro – Persono resanigita skribas al S-o P. Mauries: « VI ESTAS VERE BOUFARANTO DE HOMARO ; se via broŝuro estus tre disvastigita, ĝi faros NEKALKULEBLAN SERVON AL HOMA RASO »."
4,"Baldaŭ alvenis ankaŭ la loka fajrobrigado kun volontuloj. Ili ruliĝis tute proksimen al la fajro, rapide elsaltis el la aŭto kaj fine ili sukcesis estingi la fajron."
5,"Poloj atingis la montopinton per telfero, ĉeĥoj per piede, kaj ĉiuj renkontiĝis supre antaŭ la gastejo kie ili trinkis, manĝis kaj babilis kiel amiko kun amiko."
6,"1. Kiam iu estas prudenta, tial ke li havas la sincerecon, tiam estas dirite ke li estas tia pro naturo. Kiam iu atingis la sincerecon, tial ke li havas la prudenton, tiam estas dirite ke li estas tia pro instruo."
7,La teksto disponeblas laŭ la permesilo Krea Komunaĵo Atribuite-Samkondiĉe 3.0 Neadaptita; eble aldonaj kondiĉoj aplikeblas. Vidu la uzkondiĉojn por detaloj.
8,"Helpu al Vikipedio plilongigi ĝin. Se jam ekzistas alilingva samtema artikolo pli disvolvita, traduku kaj aldonu el ĝi (menciante la fonton)."
9,"Tiu tutmondigo kondukas al profundiĝo de abismo inter riĉaj kaj malriĉaj tavoloj de la monda loĝantaro, ĉu en riĉaj landoj, ĉu en malriĉaj landoj."


In [ ]:
ds["train"][0]

{'text': 'Ĉu ... preĝi | mediti | ricevi instigojn || kanti | muziki || informiĝi | legi | studi || prepari Diservon'}

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(model_dir, max_len=512)

In [ ]:
ds_enc = ds.map(lambda x: tokenizer(x["text"], truncation=True))

In [ ]:
ds_enc.save_to_disk("data/oscar.eo.ds")

In [ ]:
ds_enc = DatasetDict.load_from_disk("data/oscar.eo.ds")
ds_enc

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'text'],
        num_rows: 974616
    })
})

In [ ]:
tokenizer.decode(ds_enc["train"][0]["input_ids"])

'<s>Ĉu... preĝi | mediti | ricevi instigojn || kanti | muziki || informiĝi | legi | studi || prepari Diservon</s>'

## Baseline model

In [ ]:
config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
def baseline_init():
    return RobertaForMaskedLM(config=config)

In [ ]:
sample_ds = ds_enc["train"].train_test_split(train_size=512, test_size=128, seed=42)
bs = 8
logging_steps = sample_ds["train"].num_rows // bs // 4

training_args = TrainingArguments(
    output_dir="models/esperberto",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=bs,
    prediction_loss_only=True,
    evaluation_strategy="steps",
    disable_tqdm=False,
    logging_steps=logging_steps
)

trainer = Trainer(
    model_init=baseline_init,
    args=training_args,
    data_collator=data_collator,
    train_dataset=sample_ds["train"],
    eval_dataset=sample_ds["test"]
)

trainer.train();

Loading cached split indices for dataset at data/oscar.eo.ds/train/cache-8b96f05ff2fc9096.arrow and data/oscar.eo.ds/train/cache-e43a2b26405c9c65.arrow


Step,Training Loss,Validation Loss
16,10.374800,9.909284
32,9.829200,9.645468
48,9.572800,9.329382
64,9.437400,9.316128


## Custom model

Goal: implement RoBERTa LM from scratch :) Remove as much boilerplate as possible while maintaining compatibility with the trainer.

In [ ]:
from transformers import RobertaModel

In [ ]:
from transformers.models.roberta.modeling_roberta import (
    RobertaPreTrainedModel, RobertaLMHead, RobertaEmbeddings, RobertaEncoder,
    BaseModelOutputWithPoolingAndCrossAttentions)

In [ ]:
class MyRobertaModel(RobertaPreTrainedModel):

    def __init__(self, config, add_pooling_layer=True):
        super().__init__(config)
        self.config = config
        self.embeddings = RobertaEmbeddings(config)
        self.encoder = RobertaEncoder(config)
        self.pooler = RobertaPooler(config) if add_pooling_layer else None
        self.init_weights()
        
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict            
        input_shape = input_ids.size()
        batch_size, seq_length = input_shape

        device = input_ids.device 

        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape, device)


        embedding_output = self.embeddings(
            input_ids=input_ids,
        )
        encoder_outputs = self.encoder(
            embedding_output,
            attention_mask=extended_attention_mask,
            return_dict=return_dict,
        )
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output) if self.pooler is not None else None

        if not return_dict:
            return (sequence_output, pooled_output) + encoder_outputs[1:]

        return BaseModelOutputWithPoolingAndCrossAttentions(
            last_hidden_state=sequence_output,
            pooler_output=pooled_output,
            past_key_values=encoder_outputs.past_key_values,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
            cross_attentions=encoder_outputs.cross_attentions,
        )

In [ ]:
class MyRobertaForMaskedLM(RobertaPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)
        self.roberta = MyRobertaModel(config, add_pooling_layer=False)
        self.lm_head = RobertaLMHead(config)
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        labels=None,
        return_dict=None, # get rid of this
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            return_dict=return_dict,
        )
        sequence_output = outputs[0]
        prediction_scores = self.lm_head(sequence_output)

        masked_lm_loss = None
        loss_fct = CrossEntropyLoss()
        masked_lm_loss = loss_fct(
            prediction_scores.view(-1, self.config.vocab_size), labels.view(-1))

        return MaskedLMOutput(
            loss=masked_lm_loss,
            logits=prediction_scores,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [ ]:
def model_init():
    return MyRobertaForMaskedLM(config=config)

In [ ]:
# reference values

# Step	Training Loss	Validation Loss
# 16	10.272600	9.798569
# 32	9.720300	9.541902
# 48	9.513300	9.259138
# 64	9.403300	9.244128

In [ ]:
def run_trainer():
    trainer = Trainer(
    model_init=model_init,
    args=training_args,
    data_collator=data_collator,
    train_dataset=sample_ds["train"],
    eval_dataset=sample_ds["test"]
    )
    
    trainer.train()
    
run_trainer()

Step,Training Loss,Validation Loss
16,10.272600,9.798569
32,9.720300,9.541902
48,9.513300,9.259138
64,9.403300,9.244128
